# Cart Abandonment 

In [2]:
import pandas as pd
import pandasql as ps
from pandasql import sqldf

In [3]:
customer_table = pd.read_csv("customer_table.csv")
customer_table.head()

,customer_id,customer_name,age,gender,city
0,1,Customer 1,49,Female,London
1,2,Customer 2,45,Male,London
2,3,Customer 3,51,Male,London
3,4,Customer 4,38,Male,New York
4,5,Customer 5,26,Male,London


In [4]:
date_table = pd.read_csv("date_table.csv")
date_table.head()

,date_id,date
0,1,1/1/2023
1,2,1/2/2023
2,3,1/3/2023
3,4,1/4/2023
4,5,1/5/2023


In [5]:
device_table = pd.read_csv("device_table.csv")
device_table.head()

,device_id,device_type,os
0,1,Tablet,iOS
1,2,Desktop,iOS
2,3,Mobile,Windows
3,4,Mobile,Android
4,5,Tablet,iOS


In [6]:
fact_table = pd.read_csv("fact_table.csv")
fact_table.head()

,session_id,customer_id,product_id,device_id,date_id,quantity,abandonment_time
0,1,979,20,5,252,4,7/13/2023
1,2,373,12,3,353,2,11/7/2023
2,3,32,3,4,286,2,3/3/2023
3,4,631,25,1,365,2,NaN
4,5,127,9,3,163,3,7/22/2023


In [7]:
product_table = pd.read_csv("product_table.csv")
product_table.head()

,product_id,product_name,category,price
0,1,Laptop,Electronics,1026.94
1,2,Smartphone,Electronics,641.94
2,3,Headphones,Electronics,866.21
3,4,Smartwatch,Electronics,1221.77
4,5,Camera,Electronics,1290.76


### Understanding the data

In [8]:
totalCustomers = sqldf("select count(*) from customer_table")
print("totalCustomers: ",totalCustomers['count(*)'][0])

totalCustomers:  1000


In [9]:
femaleCustomers = sqldf("select count(*) from customer_table where gender='Female'")
maleCustomers = totalCustomers - femaleCustomers
print("Male Customers: ", maleCustomers['count(*)'][0])
print("Female Customers: ", femaleCustomers['count(*)'][0])

Male Customers:  499
Female Customers:  501


In [10]:
#age of customers
leastAge = sqldf("select age from customer_table order by age limit 1")
greatestAge = sqldf("select age from customer_table order by age desc limit 1")
print("minimum age of customer: ", leastAge['age'][0])
print("maximum age of customer: ", greatestAge['age'][0])

minimum age of customer:  18
maximum age of customer:  64


In [11]:
#categorizing age of customers
ageCategories = sqldf("""select
                      sum(case when age between 18 and 20 then 1 else 0 end) as teen,
                      sum(case when age between 21 and 24 then 1 else 0 end) as ya,
                      sum(case when age between 25 and 39 then 1 else 0 end) as adult,
                      sum(case when age between 40 and 64 then 1 else 0 end) as old
                      from customer_table""", locals())
ageCategories

,teen,ya,adult,old
0,70,81,296,553


Maximum customers are older people followed by adults, while less than 100 each are teen and young adult customers. 

In [12]:
#city they are from

city = sqldf("select count(*) as count, city from customer_table group by city")
city

,count,city
0,206,Berlin
1,187,London
2,185,Mumbai
3,209,New York
4,213,Sydney


In [13]:
allCategories = sqldf("""SELECT 
    city,
    gender,
    SUM(CASE WHEN age BETWEEN 18 AND 20 THEN 1 ELSE 0 END) AS teen,
    SUM(CASE WHEN age BETWEEN 21 AND 24 THEN 1 ELSE 0 END) AS young_adult,
    SUM(CASE WHEN age BETWEEN 25 AND 39 THEN 1 ELSE 0 END) AS adult,
    SUM(CASE WHEN age BETWEEN 40 AND 64 THEN 1 ELSE 0 END) AS older_adult
FROM customer_table
GROUP BY city, gender
""")
allCategories

,city,gender,teen,young_adult,adult,older_adult
0,Berlin,Female,14,9,29,56
1,Berlin,Male,6,8,32,52
2,London,Female,6,7,27,58
3,London,Male,4,8,26,51
4,Mumbai,Female,4,1,29,53
5,Mumbai,Male,9,9,31,49
6,New York,Female,7,10,32,53
7,New York,Male,6,9,35,57
8,Sydney,Female,7,8,27,64
9,Sydney,Male,7,12,28,60


### Understanding the devices

In [14]:
devices = sqldf("select count(*) from device_table")
devices

,count(*)
0,5


In [15]:
deviceTypes = sqldf("select count(*) as count, device_type from device_table group by device_type")
deviceTypes

,count,device_type
0,1,Desktop
1,2,Mobile
2,2,Tablet


In [16]:
device_table = sqldf("select * from device_table")
device_table

,device_id,device_type,os
0,1,Tablet,iOS
1,2,Desktop,iOS
2,3,Mobile,Windows
3,4,Mobile,Android
4,5,Tablet,iOS


Hence, there are 3 device types - desktop, mobile and tablet

Mobiles run in windws and android

desktop runs only on iOS

tablet runs on iOS 

In [17]:
date_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date_id  366 non-null    int64 
 1   date     366 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.8+ KB


In [18]:
date_table['date'][0]

'1/1/2023'

In [19]:
date_table['date'][365]

'1/1/2024'

In [20]:
date_table

,date_id,date
0,1,1/1/2023
1,2,1/2/2023
2,3,1/3/2023
3,4,1/4/2023
4,5,1/5/2023
...,...,...
361,362,12/28/2023
362,363,12/29/2023
363,364,12/30/2023
364,365,12/31/2023


In [21]:
distinctDates = sqldf("select count(distinct date) from date_table")
distinctDates

,count(distinct date)
0,366


There are 366 dates ranging from 1/1/2023 to 1/1/2024

i.e the data is for 1 whole year (2023)

In [22]:
fact_table.count()

session_id          5000
customer_id         5000
product_id          5000
device_id           5000
date_id             5000
quantity            5000
abandonment_time    2524
dtype: int64

In [23]:
fact_table = sqldf("select * from fact_table where abandonment_time !='NaN'")
fact_table.count()

session_id          2524
customer_id         2524
product_id          2524
device_id           2524
date_id             2524
quantity            2524
abandonment_time    2524
dtype: int64

In [24]:
countOfcustomer = sqldf("select count(*) as count, customer_id from fact_table group by customer_id")
countOfcustomer

,count,customer_id
0,2,1
1,2,2
2,1,3
3,3,4
4,1,5
...,...,...
920,1,996
921,1,997
922,1,998
923,1,999


In [25]:
countOfcustomer['count'].mean()

2.7286486486486488

In [26]:
countOfcustomer['count'].max()

8

In [27]:
abandonment = sqldf("select distinct abandonment_time from fact_table order by abandonment_time")
abandonment

,abandonment_time
0,1/1/2023
1,1/1/2024
2,1/10/2023
3,1/11/2023
4,1/12/2023
...,...
360,9/5/2023
361,9/6/2023
362,9/7/2023
363,9/8/2023


In [28]:
maxQuantityAbandoned = sqldf("select * from fact_table order by quantity desc limit 1")
maxQuantityAbandoned

,session_id,customer_id,product_id,device_id,date_id,quantity,abandonment_time
0,6,166,8,1,111,5,10/6/2023


### Analyzing further in products

In [29]:
categoryList = sqldf("select distinct category from product_table")
categoryList

,category
0,Electronics
1,Apparel
2,Home & Kitchen
3,Beauty & Personal Care
4,Sports & Outdoors


In [30]:
#no. of items in each category
items = sqldf("select group_concat(product_name), category from product_table group by category")
items

,group_concat(product_name),category
0,"T-Shirt,Jeans,Jacket,Sneakers,Dress",Apparel
1,"Shampoo,Skincare Set,Makeup Kit,Perfume,Body L...",Beauty & Personal Care
2,"Laptop,Smartphone,Headphones,Smartwatch,Camera",Electronics
3,"Blender,Cookware Set,Vacuum Cleaner,Air Purifi...",Home & Kitchen
4,"Bicycle,Yoga Mat,Dumbbells,Running Shoes,Tenni...",Sports & Outdoors


In [31]:
prod = sqldf("select count(product_name), category from product_table group by category")
prod

,count(product_name),category
0,5,Apparel
1,5,Beauty & Personal Care
2,5,Electronics
3,5,Home & Kitchen
4,5,Sports & Outdoors


In [32]:
max_cost_table = sqldf("select max(price) as max_price, product_name,product_id, category from product_table group by category order by product_id")
max_cost_table

,max_price,product_name,product_id,category
0,1290.76,Camera,5,Electronics
1,1316.50,Dress,10,Apparel
2,1450.64,Cookware Set,12,Home & Kitchen
3,1375.89,Makeup Kit,18,Beauty & Personal Care
4,799.41,Running Shoes,24,Sports & Outdoors


In [33]:
items_abandoned = sqldf("select sum(quantity) as quantity, product_id from fact_table where product_id in (select product_id from max_cost_table) group by product_id")
items_abandoned

,quantity,product_id
0,302,5
1,370,10
2,284,12
3,271,18
4,290,24


In [34]:
max_price_table = sqldf("""select b.quantity * a.max_price as cost, a.product_id, a.category, a.product_name from max_cost_table a 
                        join items_abandoned b 
                        on a.product_id = b.product_id order by cost desc""")
max_price_table

,cost,product_id,category,product_name
0,487105.00,10,Apparel,Dress
1,411981.76,12,Home & Kitchen,Cookware Set
2,389809.52,5,Electronics,Camera
3,372866.19,18,Beauty & Personal Care,Makeup Kit
4,231828.90,24,Sports & Outdoors,Running Shoes


In each category, just picking the most expensive abandoned products. 

For apparel, dress abandonment alone costs 487,105 units of price.

1. finding the total cost abandoned in each category

2. finding device and customer segment that abandons

### Finding the total cost abandoned in each category

In [35]:
price = sqldf("""SELECT 
                    p.category, 
                    p.product_name, 
                    SUM(f.quantity) as sum_quantity,
                    SUM(f.quantity) * p.price AS total_price_for_product
                FROM fact_table f
                JOIN product_table p ON f.product_id = p.product_id
                GROUP BY p.category, p.product_name, p.price""")
price

,category,product_name,sum_quantity,total_price_for_product
0,Apparel,Dress,370,487105.00
1,Apparel,Jacket,352,156485.12
2,Apparel,Jeans,288,347310.72
3,Apparel,Sneakers,291,378922.74
4,Apparel,T-Shirt,278,252040.36
5,Beauty & Personal Care,Body Lotion,235,298898.85
6,Beauty & Personal Care,Makeup Kit,271,372866.19
7,Beauty & Personal Care,Perfume,309,36767.91
8,Beauty & Personal Care,Shampoo,252,251879.04
9,Beauty & Personal Care,Skincare Set,276,200958.36


In [36]:
total_category = sqldf("select sum(total_price_for_product) as total_price, category from price group by category order by total_price desc")
total_category

,total_price,category
0,1645995.38,Electronics
1,1621863.94,Apparel
2,1161370.35,Beauty & Personal Care
3,1048704.66,Home & Kitchen
4,698547.62,Sports & Outdoors


The most costliest category abandoned is electronics, closely followed by apparel. 

In [37]:
total_quantity = sqldf("select sum(sum_quantity) as s_quantity, category from price group by category order by s_quantity desc")
total_quantity

,s_quantity,category
0,1651,Electronics
1,1579,Apparel
2,1494,Home & Kitchen
3,1454,Sports & Outdoors
4,1343,Beauty & Personal Care


No. of products abandoned are almost equal 

### Finding the customer segment that abandons cart the most

In [46]:
abandon_segment = sqldf("""select 
                    case 
                        when c.age between 18 and 20 then 'Teen'
                        when c.age between 21 and 24 then 'YA'
                        when c.age between 25 and 39 then 'Adult'
                        when c.age between 40 and 64 then 'Older Adult'
                    end as age_segment, 
                    sum(f.quantity)/sum() as total_products_abandoned
                    from customer_table c join fact_table f on c.customer_id = f.customer_id
                    where f.abandonment_time !='NaN'
                    group by age_segment""")
abandon_segment

,age_segment,total_products_abandoned
0,Adult,2197
1,Older Adult,4222
2,Teen,507
3,YA,595


#### The older adults abandoned the most products 
But the number of older adults in the customer segment is large, hence let's find the average abandonment in each segment 

In [63]:
df = sqldf("""SELECT 
                CASE 
                    WHEN c.age BETWEEN 18 AND 20 THEN 'Teen'
                    WHEN c.age BETWEEN 21 AND 24 THEN 'Young Adult'
                    WHEN c.age BETWEEN 25 AND 39 THEN 'Adult'
                    WHEN c.age BETWEEN 40 AND 64 THEN 'Older Adult'
                    ELSE 'Senior' 
                    end as age_segment,
                    count(distinct c.customer_id) AS total_customers,
                    count(distinct f.customer_id) AS abandoning_customers,
                    count(distinct f.customer_id) *1.0/ count(distinct c.customer_id) AS abandonment_proportion
                FROM customer_table c
                LEFT JOIN fact_table f ON c.customer_id = f.customer_id AND f.abandonment_time IS NOT NULL
                GROUP BY age_segment
                ORDER BY abandonment_proportion DESC;
            """)
df

,age_segment,total_customers,abandoning_customers,abandonment_proportion
0,Young Adult,81,77,0.950617
1,Older Adult,553,513,0.927667
2,Adult,296,271,0.915541
3,Teen,70,64,0.914286


In [64]:
proportion = [
    abandon_segment['total_products_abandoned'][0] / ageCategories['adult'].values[0],
    abandon_segment['total_products_abandoned'][1] / ageCategories['old'].values[0],
    abandon_segment['total_products_abandoned'][2] / ageCategories['teen'].values[0],
    abandon_segment['total_products_abandoned'][3] / ageCategories['ya'].values[0]
]

print(proportion)

[7.422297297297297, 7.634719710669078, 7.242857142857143, 7.345679012345679]


#### The abandonment_proportion in each segment approximately equals 90% and young adults are in alarming 95% compared to 91% in teen and adult 

#### The average products abandoned by the customers in each segment is around 7, with older adults leaning to 7.6

In [76]:
# probing device types
device_abandoned = sqldf("""select
                            d.device_type,
                            d.os,
                            sum(f.quantity) as quantity_abandoned
                            from device_table d inner join fact_table f
                            on d.device_id = f.device_id
                            group by d.device_type, d.os
                            """)
device_abandoned

,device_type,os,quantity_abandoned
0,Desktop,iOS,1525
1,Mobile,Android,1480
2,Mobile,Windows,1521
3,Tablet,iOS,2995


Tablet users have abandoned the most quantity. Let's find the customer proportion

In [85]:
proportion_devices = sqldf("""select
                                d.device_type, 
                                d.os,
                                count(distinct(c.customer_id)) as customers
                                from device_table d
                                inner join fact_table f on d.device_id=f.device_id
                                inner join customer_table c on f.customer_id = c.customer_id
                                group by d.device_type, d.os
                            """)
proportion_devices

,device_type,os,customers
0,Desktop,iOS,405
1,Mobile,Android,392
2,Mobile,Windows,398
3,Tablet,iOS,636


In [91]:
[device_abandoned['quantity_abandoned'][i]/proportion_devices['customers'][i] for i in range(4)]

[3.765432098765432, 3.7755102040816326, 3.821608040201005, 4.709119496855346]

#### The tablet customers end up abandoning more.

In [95]:
city_abandonment = sqldf("""
                            SELECT 
                                c.city,
                                COUNT(DISTINCT c.customer_id) AS total_customers,
                                COUNT(DISTINCT f.customer_id) AS abandoning_customers,
                                COUNT(DISTINCT f.customer_id) * 1.0 / COUNT(DISTINCT c.customer_id) AS abandonment_rate
                            FROM customer_table c
                            LEFT JOIN fact_table f ON c.customer_id = f.customer_id 
                                AND f.abandonment_time IS NOT NULL
                            GROUP BY c.city
                            ORDER BY abandonment_rate DESC
                        """)
city_abandonment

,city,total_customers,abandoning_customers,abandonment_rate
0,Berlin,206,195,0.946602
1,Sydney,213,199,0.934272
2,London,187,171,0.914439
3,New York,209,191,0.913876
4,Mumbai,185,169,0.913514


The cities have approximately same abandonment rate

## Customer Demographics & Behavior
Total Customers: Identified the number of customers, with a gender split.

Age Distribution: Most customers are older adults (40-64 years), followed by adults (25-39 years). Teens (18-20) and young adults (21-24) form a smaller segment.

City-Wise Distribution: Customers are spread across multiple cities.

### Device Insights
Three device types: Desktop, Mobile, and Tablet.

Mobiles run on Android & Windows, desktops on iOS, and tablets on iOS.

Tablet users abandon the most products in proportion to their numbers.

### Product Insights
Electronics and Apparel are the most abandoned categories in terms of cost.

Highest value abandonment: Dress abandonment alone costs ₹487,105.

Total number of products abandoned is almost equal across categories.

#### Customer Segments with Highest Abandonment Rates
Older adults (40-64) abandoned the most products in total count.

However, young adults (21-24) have the highest abandonment proportion (95%), followed by teens (91%).

On average, customers abandon ~7 products, with older adults leading at 7.6.

#### Device & OS Impact on Abandonment
Tablet users have the highest abandonment rates.

Windows & Android mobile users also contribute significantly.

Abandonment rates are consistent across different cities.

### Key Takeaways
Electronics and Apparel need better cart retention strategies (e.g., discounts, reminders).

Young adults & tablet users are high-priority segments for reducing abandonment.

City-based marketing strategies may not significantly reduce abandonment since rates are uniform.